In [ ]:
import os

In [ ]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml\\AI-powered-Bank-Product-Recommender-Chatbot'

In [ ]:
os.chdir("../")

In [ ]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [ ]:
project_dir = "C:/Users/RICH-FILES/Desktop/ml/AI-powered-Bank-Product-Recommender-Chatbot"
os.chdir(project_dir)

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    raw_data_dir: Path
    db_file: Path
    customers_table: str
    products_table: str
    customers_csv: Path
    products_csv: Path
    

In [ ]:
from BankProducts.constants import *
from BankProducts.utils.common import read_yaml, create_directories

In [ ]:
# create configuration manager 
class ConfigurationManager: 
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH, 
        schema_filepath = SCHEMA_FILE_PATH,
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        db  = self.config.database
        
        
        create_directories([config.root_dir], [config.raw_data])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            raw_data_dir=config.raw_data_dir,
            db_file = db.db_file,
            customers_table = db.customers_table,
            products_table = db.products_table,
            customers_csv = config.customers_csv,
            products_csv = config.products_csv
            
        )
        
        return data_ingestion_config

In [ ]:
import pandas as pd
import sqlite3
from pathlib import Path
from BankProducts.utils.common import read_yaml, create_directories, save_csv_file
from BankProducts import logger


ImportError: cannot import name 'save_csv_file' from 'BankProducts.utils.common' (C:\Users\RICH-FILES\Desktop\ml\AI-powered-Bank-Product-Recommender-Chatbot\src\BankProducts\utils\common.py)

In [ ]:
class DataIngestion:
    def__init__(
        self, config: DataIngestionConfig):
        self.config = config
        
    def extract_and_save_data(self):
        logging.info(f"Connecting to database: {self.config.db_file}")
        conn = sqlite3.connect(self.config.db_file)

        try:
            logging.info(f"Reading table: {self.config.customers_table}")
            customers_df = pd.read_sql_query(f"SELECT * FROM {self.config.customers_table}", conn)

            logging.info(f"Reading table: {self.config.products_table}")
            products_df = pd.read_sql_query(f"SELECT * FROM {self.config.products_table}", conn)
        finally:
            conn.close()
            logging.info("Database connection closed.")
            
        # Create directories
        create_directories([self.config.raw_data_dir])

        # Save files
        customers_df.to_csv(self.config.customers_csv, index=False)
        products_df.to_csv(self.config.products_csv, index=False)
        logging.info(f"Saved customers to: {self.config.customers_csv}")
        logging.info(f"Saved products to: {self.config.products_csv}")


In [ ]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.extract_and_save_data()
except Exception as e:
    logger.exception(e)
    raise e
